1. most used platform
2. no. of failed answers per conversation and no. of questions that did not satisfy user (with feedback no)
3. questions that the bot couldn't answer/ got neg feedback  
4. character length of user-requests 
5. perform NER and find commonly asked topics 
6. Do coreferences exist?

In [3]:
import json
import pprint
import re

In [4]:
path = 'data/grouped_chats.json'

with open(path, 'r') as file:
    data = json.load(file)
    i = 0
    for key, value in data.items():
        print(len(value))
        pprint.pprint(value)
        i += 1
        if i == 5:
            break

24
[{'fromBot': False,
  'id': 'dbfaba55-ecc0-42a3-b2c7-1daa39fb36b3_1553869389296_request',
  'request': {'messageData': [{'payload': 'start'}],
              'messageType': 'parameter',
              'platform': 'Web',
              'userId': 'dbfaba55-ecc0-42a3-b2c7-1daa39fb36b3'},
  'response': nan,
  'timestamp': 1553869389296.0,
  'userId': 'dbfaba55-ecc0-42a3-b2c7-1daa39fb36b3'},
 {'fromBot': True,
  'id': 'dbfaba55-ecc0-42a3-b2c7-1daa39fb36b3_1553869389296_response',
  'request': nan,
  'response': [{'messageData': [{'buttons': [],
                                 'fields': [],
                                 'image': 'http://i.imgur.com/044oUKK.png',
                                 'smartReplies': [],
                                 'text': 'I am the DBpedia Bot'}],
                'messageType': 'text'},
               {'messageData': [{'buttons': [],
                                 'fields': [],
                                 'image': 'http://i.imgur.com/044oUKK.png',


 {'fromBot': True,
  'id': '448e21ab-e862-42d3-b106-50608fea9381_1515752315159_response',
  'request': nan,
  'response': [{'messageData': [{'buttons': [],
                                 'fields': [],
                                 'image': 'http://i.imgur.com/044oUKK.png',
                                 'smartReplies': [],
                                 'text': 'DBpedia is a crowd-sourced community '
                                         'effort to extract structured '
                                         'information from Wikipedia and make '
                                         'this information available on the '
                                         'Web.'}],
                'messageType': 'text'},
               {'messageData': [{'buttons': [{'buttonType': 'link',
                                              'slackStyle': 'default',
                                              'title': 'Learn More',
                                              'uri': 'htt

 {'fromBot': True,
  'id': '448e21ab-e862-42d3-b106-50608fea9381_1516765307073_response',
  'request': nan,
  'response': [{'messageData': [{'buttons': [],
                                 'fields': [],
                                 'image': 'http://i.imgur.com/044oUKK.png',
                                 'smartReplies': [],
                                 'text': 'This is what I found:'}],
                'messageType': 'text'},
               {'messageData': [{'buttons': [{'buttonType': 'link',
                                              'slackStyle': 'default',
                                              'title': 'View Map',
                                              'uri': 'http://nominatim.openstreetmap.org/search?q=Berlin&addressdetails=1&extratags=1&limit=1'},
                                             {'buttonType': 'param',
                                              'slackStyle': 'default',
                                              'title': 'Basic Informa

                                              'uri': 'dbpedia-contribute'},
                                             {'buttonType': 'param',
                                              'slackStyle': 'default',
                                              'title': 'Is DBpedia down?',
                                              'uri': 'check_service__dbpedia'}],
                                 'fields': [],
                                 'image': '/images/icon-dbpedia-92.png',
                                 'smartReplies': [],
                                 'text': 'You can ask me questions related to '
                                         'DBpedia such as:',
                                 'title': 'About DBpedia'},
                                {'buttons': [{'buttonType': 'param',
                                              'slackStyle': 'default',
                                              'title': 'Who is Einstein?',
                                       

               {'messageData': [{'buttons': [{'buttonType': 'link',
                                              'slackStyle': 'default',
                                              'title': 'View Map',
                                              'uri': 'http://nominatim.openstreetmap.org/search?q=India&addressdetails=1&extratags=1&limit=1'},
                                             {'buttonType': 'param',
                                              'slackStyle': 'default',
                                              'title': 'Basic Information',
                                              'uri': 'entity-information__http://dbpedia.org/resource/India'}],
                                 'fields': [{'isShort': True,
                                             'name': 'Country',
                                             'value': 'India'},
                                            {'isShort': True,
                                             'name': 'Latitude',
     

  'response': [{'messageData': [{'buttons': [{'buttonType': 'link',
                                              'slackStyle': 'default',
                                              'title': 'View in Wikipedia',
                                              'uri': 'http://en.wikipedia.org/wiki/German_Empire'},
                                             {'buttonType': 'link',
                                              'slackStyle': 'default',
                                              'title': 'View in DBpedia',
                                              'uri': 'http://dbpedia.org/resource/German_Empire'},
                                             {'buttonType': 'param',
                                              'slackStyle': 'default',
                                              'title': 'Learn More',
                                              'uri': 'learn-more__http://dbpedia.org/resource/German_Empire__German '
                                              

In [5]:
# most used platform 
from collections import Counter

platform_count = Counter()
err = 0

for key, val in data.items():
    msg = val[0]
    if type(msg['request']) == float: # check if msg.request = nan
        err += 1
#         pprint.pprint(msg)
    else:
        platform_count[msg['request']['platform']] += 1
            
print(platform_count)
print(err)

Counter({'Web': 9078, 'Slack': 4, 'Facebook Messenger': 1})
1


In [13]:
# no. of failed answers per conversation and questions that bot couldn't answer
# calculate how long the conv persists after first neg feedback per user 
from collections import defaultdict
from collections import Counter

failed_qid_per_user = defaultdict(set)
failed_qid = set()
num_requests = 0 # includes START payload from user  
passed = 0
feedback_asked = 0 
conv_len_negFeed = Counter() # {uid: number of dialogues after 1st neg feedback}
has_uid_negFeed = defaultdict(lambda : False) # {uid: Boolean value for whether neg feedback was rcvd}

for uid, conv in data.items():
    for msg in conv: # msg object
        if not msg['fromBot']: 
            num_requests += 1
            for obj in msg['request']['messageData']:
                if 'payload' in obj and 'feedback__no__' in obj['payload']:
                    if not has_uid_negFeed[uid]:
                        has_uid_negFeed[uid] = True
                        conv_len_negFeed[uid] += 1
                    else:
                        conv_len_negFeed[uid] += 1
                    failed_qid_per_user[uid].add(obj['payload'].replace('feedback__no__', ''))
                    failed_qid.add(obj['payload'].replace('feedback__no__', ''))
                    break
                elif 'payload' in obj and 'feedback__yes__' in obj['payload']:
                    passed += 1
                    break
                elif 'text' in obj and has_uid_negFeed[uid]:
                    conv_len_negFeed[uid] += 1
                
        else:
            if type(msg['response']) != float:
                for obj in msg['response']:
                    if obj['messageType'] == 'smart_reply':
                        feedback_asked +=1
                        break


print('num_requests: ', num_requests)
print('feedback_no: ', len(failed_qid))
print('convos with failed responses: ', len(failed_qid_per_user))
print('feedback_yes: ', passed)
print('feedback_asked: ', feedback_asked)
print('has_uid_negFeed: ', has_uid_negFeed, len(has_uid_negFeed))
print('conv_len_negFeed: ', conv_len_negFeed)

num_requests:  45797
feedback_no:  4155
convos with failed responses:  1982
feedback_yes:  3406
feedback_asked:  28953
has_uid_negFeed:  defaultdict(<function <lambda> at 0x7f91a755ad90>, {'dbfaba55-ecc0-42a3-b2c7-1daa39fb36b3': True, '448e21ab-e862-42d3-b106-50608fea9381': True, 'cded1ee9-9196-43cc-810e-fc9b09b35ed6': True, '03b12284-8cc5-47e2-9786-2319ee6b32ea': True, 'xqmzefkegm': False, '856d47d1-08e1-4c00-84e5-f808de8e7e0c': False, '2649bb03-1822-42f8-8eeb-d4589e357937': False, '2136afd1-2d32-4c4b-9220-69d62bfc882d': True, 'd52b7e87-f5b0-474a-95a6-74c7e20fba88': True, '82c3076e-ad3f-4ee5-ad5c-3c174697a193': True, '71d1e826-d212-4a40-9038-afb6df58c72b': True, 'd54d7b1f-1639-4baa-a142-200c725d22b5': False, '52ecd3a7-9d18-43aa-92ad-fd59c8699902': True, 'a60323b9-ebf9-4ac6-b1cb-ef441f2dc7fa': True, '4f83c6bd-cbe6-4f59-8605-a68c30e6075d': True, '630d4681-33be-4084-b26d-27f95cebef84': True, '7db9ee4b-70a4-4d4e-b089-e54b2515ac07': True, '3cced261-002d-4730-83b6-537b11c687a5': False, '28a

In [14]:
neg = 0
no_or_pos = 0
for k, v in has_uid_negFeed.items():
    if v==False:
        no_or_pos += 1
    else:
        neg += 1
print(neg)
print(no_or_pos)

1982
5519


In [19]:
print(len(conv_len_negFeed))
from statistics import mean 
import operator
print(mean(conv_len_negFeed.values()))
print(sorted(conv_len_negFeed.values(), reverse=True))

1982
6.899596367305752
[604, 137, 127, 121, 94, 93, 92, 90, 82, 76, 72, 67, 65, 59, 57, 54, 50, 47, 47, 46, 46, 44, 44, 43, 43, 42, 41, 40, 39, 38, 38, 38, 38, 37, 36, 36, 36, 35, 34, 33, 33, 32, 31, 31, 31, 31, 30, 29, 29, 29, 28, 28, 27, 26, 26, 26, 25, 25, 25, 25, 25, 25, 24, 24, 24, 24, 24, 24, 24, 23, 23, 23, 23, 23, 23, 22, 22, 22, 22, 22, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 17, 17, 17, 17, 17, 17, 17, 17, 17, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 

In [4]:
# questions that the bot couldn't answer
unanswered_ques = []

for uid, conv in data.items():
    if uid in failed_qid_per_user:
        for msg in conv: # msg object
            if not msg['fromBot']:
                if msg['id'].replace('_request', '') in failed_qid_per_user[uid]:
                    for obj in msg['request']['messageData']:
#                         print(obj)
                        if 'text' in obj:
#                             print(obj)
                            unanswered_ques.append(obj['text'])
                            #break

print(len(unanswered_ques))

3986


In [7]:
print(unanswered_ques)
with open('data/unanswered_req', 'w') as f:
    f.write(unanswered_ques)

['Who is George MacDonald Church ?', 'Who is Macron ?', 'What is the plot of Titanic?', 'who is sachin>', 'what is anthem of india?', 'what is life', 'How many bones does a cat have?', 'How to care a newborn?', 'How to care for newborn?', 'Where was Angela Merkel born ', 'who is Angela Merkel?', 'Tell me something about Rachmaninov', 'what is konami', 'hello', 'where was george clooney born?', 'who is the mayor of Varese?', 'What is the German Empire?', 'who is einstein', 'Programming Language', 'how many members does the Beatles have?', 'The top biggest cities', 'The most difficult problem ', 'heloo', 'what is apple', 'Abbeville County', 'which country has the largest population', 'what is the population of earth?', 'how much bigger is kolkata than mumbai?', 'what is a green cloudy object in space discovered by a girl in the netherlands', 'green space cloud', 'green object in space', 'How do you work?', 'How do you work?', 'How many children had alber einstein', 'what year did star wa

TypeError: write() argument must be str, not list

In [38]:
# get convos as request-response pairs, irrespective of userId  
utter = dict() # { qid : {req : [response]}}

for uid, conv in data.items():
    for msg in conv: # msg object
        if type(msg['id']) != float and '_request' in msg['id']: # request-from-user, add to dict
            qid = msg['id'].replace('_request', '')
            req_ob = msg['request']['messageData'][0]
            for key in req_ob:
                request = req_ob[key]
                obj = {request : []}
                utter[qid] = obj # every req-response pair is marked by a unique id 
                
        elif type(msg['id']) != float:
            resp_id = msg['id'].replace('_response', '')
            resp_list = msg['response']
            response = []
            if resp_id in utter:
                for obj in resp_list:
                    for text_ob in obj['messageData']:
                        if 'text' not in text_ob:
                            response.append(text_ob['title'])
                        else:
                            response.append(text_ob['text'])
                       
                for key in utter[resp_id]:
                    utter[resp_id][key].extend(response)
                    
with open('data/utterances.json', 'w') as outfile:
    json.dump(utter, outfile)


In [39]:
print(len(utter))

45797


In [40]:
# character length of user-requests 
from collections import Counter
reqlen = Counter() # {len - no. of requests having that len}

for id in utter:
    for req in utter[id]:
        if req != 'start' and 'feedback' not in req:
            reqlen[len(req)] += 1
pprint.pprint(reqlen)
print(max(reqlen.keys()))
print(min(reqlen.keys()))
from statistics import mean
print(mean(list(reqlen.keys())))


Counter({13: 1848,
         16: 1585,
         20: 1263,
         15: 1210,
         63: 1210,
         17: 1144,
         18: 1126,
         12: 989,
         22: 956,
         14: 916,
         19: 894,
         2: 859,
         54: 755,
         3: 752,
         21: 745,
         23: 666,
         5: 631,
         24: 615,
         11: 593,
         4: 588,
         25: 576,
         26: 533,
         30: 521,
         27: 517,
         31: 499,
         29: 498,
         28: 496,
         32: 389,
         6: 383,
         7: 371,
         9: 356,
         8: 352,
         44: 343,
         10: 336,
         33: 328,
         34: 306,
         47: 303,
         35: 281,
         36: 235,
         37: 225,
         38: 221,
         39: 196,
         40: 177,
         56: 166,
         42: 157,
         41: 152,
         61: 136,
         43: 124,
         72: 112,
         46: 103,
         74: 99,
         1: 87,
         68: 84,
         52: 82,
         48: 77,
         45: 76,


In [9]:
# character-length of unanswered-requests with neg feedback
from collections import Counter
reqlen = Counter() # {len - no. of requests having that len}
for ques in unanswered_ques:
    reqlen[len(ques)] += 1
pprint.pprint(reqlen)
print(max(reqlen.keys()))
print(min(reqlen.keys()))

Counter({16: 222,
         17: 173,
         15: 165,
         13: 159,
         14: 154,
         19: 148,
         20: 147,
         18: 138,
         21: 135,
         12: 125,
         22: 114,
         23: 114,
         27: 110,
         24: 104,
         25: 102,
         26: 94,
         30: 92,
         32: 90,
         2: 90,
         28: 89,
         31: 86,
         11: 79,
         29: 78,
         44: 65,
         33: 64,
         5: 54,
         47: 53,
         35: 51,
         34: 50,
         38: 49,
         3: 48,
         39: 47,
         9: 45,
         40: 42,
         36: 42,
         42: 41,
         8: 41,
         37: 39,
         41: 37,
         10: 34,
         43: 34,
         6: 33,
         4: 33,
         7: 31,
         46: 28,
         48: 22,
         61: 19,
         49: 17,
         45: 16,
         50: 14,
         52: 14,
         53: 12,
         51: 11,
         54: 8,
         55: 8,
         58: 7,
         56: 7,
         1: 5,
         74: 

#### analyse requests - perform NER and find commonly asked topics

In [4]:
# load data 
with open('data/utterances.json', 'r') as file:
    utter = json.load(file)

print(len(utter))

45797


In [5]:
def preprocess(req):
    req = re.sub(r'http://dbpedia.org/resource/', ' ', req)
    req = re.sub(r"-|_|/|!|>|\\", ' ', req) 
    req = re.sub(r"^'", '', req)
    req = re.sub(r' +', ' ', req)
    return req.strip()

In [6]:
# data cleaning
def data_clean():
    clean_text = []
    for id in utter:
        for req in utter[id]:
            if req != 'start' and 'feedback' not in req:
                clean_text.append(preprocess(req))
    return clean_text

preprocessed_text = data_clean()

In [7]:
print(len(preprocessed_text))

30263


In [8]:
# data cleaning for unanswered requests
def data_clean(all_text):
    clean_text = []
    for req in all_text:
        clean_text.append(preprocess(req))
    return clean_text

preprocessed_un_req = data_clean(unanswered_ques)

NameError: name 'unanswered_ques' is not defined

In [60]:
print(len(preprocessed_un_req))

3986


In [10]:
# perform NER
import requests
from collections import defaultdict

def get_surface_forms_schema(response, entity_map):
    sf_text = {} # surface_form:schema 
    if 'Resources' in response:
        for res in response['Resources']:
            surface_form = res['@surfaceForm']
            types = res['@types']
            if len(types) != 0:
                type_dict = dict_from_str(types) 
                if 'Schema' in type_dict:
                    for sch in type_dict['Schema']:
                        entity_map[sch].append(surface_form)
                        sf_text[surface_form] = sorted(type_dict['Schema'])[0]
    return sf_text

def dict_from_str(string): # str: 'abc:def,fgh:ijk,..'
    d = defaultdict(list)
    for element in string.split(','):
        el = element.split(':')
        d[el[0].strip()].append(el[1].strip())
    return d

def extract_entities(preprocessed_text, replace=True):
    entity_map = defaultdict(list) # schema: [entity] -> {person: [name1, ..], thing: [thing1,..]}
    i = 0
    ner_text = []
    for text in preprocessed_text:
        i +=1 
        if i%100==0:
            print(i)
        headers = {'Accept': 'application/json'}
        data = {"text":text, 'confidence': '0.35'}
        response = requests.post('http://model.dbpedia-spotlight.org/en/annotate', headers=headers, data=data)
        if response.status_code == requests.codes.ok:
            print(response)
            json_res = response.json()
            pprint.pprint(json_res)
            sf_schema = get_surface_forms_schema(json_res, entity_map)
            if replace: # replace the surface form with Schema in string
                for sf in sf_schema:
                    if not text.startswith(sf) and sf.lower() != 'dbpedia':
                        text = text.replace(sf, sf_schema[sf])
                        ner_text.append(text)
                if len(sf_schema) == 0:
                    ner_text.append(text)
            break
#     print(ner_text)
    return entity_map, ner_text

entity_map, ner_text = extract_entities(preprocessed_text,True)
# extract_entities(['美國總統'], True)

<Response [200]>
{'@confidence': '0.35',
 '@policy': 'whitelist',
 '@sparql': '',
 '@support': '0',
 '@text': 'Who is George MacDonald Church ?',
 '@types': '',
 'Resources': [{'@URI': 'http://dbpedia.org/resource/George_MacDonald',
                '@offset': '7',
                '@percentageOfSecondRank': '3.304585892431123E-7',
                '@similarityScore': '0.9999996636666989',
                '@support': '351',
                '@surfaceForm': 'George MacDonald',
                '@types': 'Http://xmlns.com/foaf/0.1/Person,Wikidata:Q5,Wikidata:Q36180,Wikidata:Q24229398,Wikidata:Q215627,DUL:NaturalPerson,DUL:Agent,Schema:Person,DBpedia:Writer,DBpedia:Person,DBpedia:Agent'}]}


In [84]:
print(len(ner_text))

27143


In [85]:
with open('ner_reqs_all.txt', 'w') as f:
    f.write(str(ner_text))
    
with open('all_entity_map', 'w') as f:
    f.write(str(entity_map))

In [86]:
print(ner_text)

['Who is Person Church ?', 'Who is Person ?', 'Who is Person ?', 'Who is Person ?', 'entity information Person', 'WHERE IS PARIS?', 'WHO IS Person?', 'WHO IS MACRON?', "Who is Person'swife？", 'entity information Person', 'entity information Planet', 'get location City', 'entity information Person', 'learn more Person Person', 'load similar Person Person', 'get location City', 'entity information Planet', 'entity information City', 'learn more Planet Planet', 'load similar Planet Planet', 'load related Planet Planet', 'entity information Person', 'get location City', 'learn more Person Person', 'who is Person', 'get location City', 'where is AdministrativeArea', 'Where is Place', 'Where is AdministrativeArea', 'What is Organization', 'What is Organization', 'entity information Planet', 'get location City', 'Who directed MusicGroup of Tomorrow?', 'What is the plot of Titanic?', 'who is sachin', 'who is sachin?', 'hello', 'what you can do?', 'where is City?', 'alright', 'yes', 'where is i

In [77]:
print(entity_map)

defaultdict(<class 'list'>, {'Person': ['George MacDonald', 'Macron', 'Angela Merkel', 'Angela Merkel', 'george clooney', 'einstein', 'einstein', 'Max Planck', 'bruno mars', 'bruno mars', 'Robin Williams', 'usain bolt', 'wittgenstein', 'wittgenstein', 'ludwig wittgenstein', 'obama', 'einstein', 'Edison', 'Cristiano Ronaldo', 'steph curry', 'René Char', 'René Char', 'Donald Trump', 'Donald Trump', 'Jacques Chirac', 'CHIRAC', 'harrison ford', 'elvis', 'einstein', 'Obama', 'Barak Obama', 'Barak Obama', 'PUTIN', 'Barack Obama', 'Barack Obama', 'Barack Obama', 'Barack Obama', 'Cras', 'Cras', 'Cras', 'Barack Obama', 'Angela Merkel', 'tintoretto', 'tintoretto', 'kamal hassan', 'Chiang Kai shek', 'Chiang Kai shek', 'Jimmy Wales', 'Albert Einstein', 'Obama', 'Barack Obama', 'einstein', 'narendra modi', 'sebastian', 'donald trump', 'J.R.R TOLKIEN', 'frank sinatra', 'Narendra Modi', 'Narendra Modi', 'Washington', 'Victor Hugo', 'elvis presley', 'drew brees', 'justin bieber', 'morgan freeman', 'ce

In [24]:
schema_entity_map = entity_map
with open('data/entity_map.json', 'w') as f:
    json.dump(entity_map, f)

In [87]:
# schema of recognised entities 
for k,v in entity_map.items():
    print(k, len(v))

Person 5661
RadioStation 51
Organization 1105
Place 5308
City 1727
CreativeWork 2561
AdministrativeArea 226
MusicGroup 350
Country 1707
Event 79
Language 159
Mountain 24
RiverBodyOfWater 30
BodyOfWater 37
Movie 113
MusicAlbum 13
LakeBodyOfWater 6
SportsTeam 112
TelevisionStation 15
LandmarksOrHistoricalBuildings 24
EducationalOrganization 106
CollegeOrUniversity 98
WebPage 46
SportsEvent 15
Product 37
BankOrCreditUnion 18
Hospital 1
Book 37
Continent 14
Airport 13
School 8
Park 7
MusicRecording 4
TVEpisode 1
SkiResort 1


In [34]:
# write to file 
with open('data/ner_requests.txt', 'w') as f:
    f.write(str(ner_text))

#### Do coreferences exist?

In [9]:
import spacy, ast
nlp = spacy.load('en_core_web_md')

import neuralcoref
neuralcoref.add_to_pipe(nlp)

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: spacy.tokens.span.Span size changed, may indicate binary incompatibility. Expected 72 from C header, got 80 from PyObject
  return f(*args, **kwds)


In [ ]:
coref = []
for line in preprocessed_text:
    doc = nlp(line)
    if doc._.has_coref:
        coref.append((line, doc._.coref_clusters))

print(len(coref))
print(coref)

In [105]:
# check the POS tag for pronoun

coref = []
has_you = []
has_me = []
pronouns = set()
you_me = []

# with open('ner_requests.txt', 'r') as f:
#     lines = f.read()
for line in preprocessed_text:
    if 'tell me' in line.lower():
        line = line.lower().replace('tell me', 'give')
        print(line)
    doc = nlp(line)
    if_you = False
    if_me = True
    has_you_me = False
    for token in doc:
        if token.pos_=='PRON' and token.tag_=='PRP':
            pronouns.add(token.text.lower())
            if token.text.lower() == 'it' or token.text.lower() == 'its' or token.text.lower() == "it's":
                coref.append(line)
            if token.text.lower() == 'you' or token.text.lower() == 'your' or token.text.lower() == 'yours':
                has_you.append(line)
                if_you = True
                if if_me and not has_you_me:
                    you_me.append(line)
            if token.text.lower() == 'me'or token.text.lower() == 'i' or token.text.lower() == 'my' or token.text.lower() == 'mine':
                has_me.append(line)
                if_me = True
                if if_you and not has_you_me:
                    you_me.append(line)
        

print('# coref: ', len(coref))
print('# times chatbot was referred: ', len(has_you))
print('# times user refers himself: ', len(has_me))
print('# reqs: ', len(preprocessed_text))
print('pronouns: ', pronouns)
            

give something about rachmaninov
give about dbpedia lookup
give about dbpedia lookup
give about obama
give about obama
give about barack obama
give about obama
give about obama
give the actors in the movie inception
give actors in game of thrones
give about dr abdul kalam
can you give what is an apple
could you give about the best disguises mortadelo has ever used?
could you give please who is the mortadelo companion?
give about rajanikanth
give about usain bolt
give the football champion in 2010
give the german history
give about einstein
can you give how you work?
give about the dbpedia bot
give about yourself
can you give about other things?
give about birds
could you please give about birds? what are they?
give about dbpedia
give about east indian company
can you give what formula one race is this weekend?
give about dbpedia
give about dbpedia
give about dbpedia lookup
give about gsoc
give about barack obama
give about barack obama
give about barack obama
could you give about barac

give my letter words
give about nba
give about trump
give about bill gates
give about sokoloff
give about chandrashekhar azad
give about vijay mallya
please, give more about yo yo ma
give about fruits
give about and about it
give about mango
give about tirumala tirupati devasthanam
give about jaggi vasudev
give about stars
give about old board
give about jayalalitha
give about indian population
can you give how to get to sesame street?
give about integral
give about fruits and their colour
give about potti sriramulu
so give information about germany
give about city of atlanta
give about china
give about coriander
give something about chhatrapati shivaji
give about mumbai
give beijin in italian
can you give the numbers from 1 to 10
give why
give about barack obama
give a panchatantra story
give about netaji
give about cow
give something about narendra modi
give about republic day
give about crow
give about der fliedermaus
give about childrens day
give about yourself
give about apple
giv

In [101]:
# kind of pronouns/coreferences used
print(pronouns)

{'she', 'ther', 'myself', 'it', 'c', 'you', 'whait', 'her', 'yo', 'hwo', 'he', 'levis', 'hp', 'em', 'him', 'y', 'u', 'hw', 'we', 'yourself', 'us', 'my', 'them', 'they', 'i', "'s", 'me', 'ut', 'yours'}


In [106]:
print(coref)

['what day is it today', 'Google is it a company ?', 'Glad that you like it.', 'Glad that you like it.', 'Glad that you like it.', 'Glad that you like it.', 'Glad that you like it.', 'Glad that you like it.', 'Glad that you like it.', 'Glad that you like it.', 'Glad that you like it.', 'Glad that you like it.', 'is it going to snow', 'how far is it to the moon', 'is it a full moon tonight', 'is it gonna be called tomorrow', 'is it time for bed', 'how far is it to work', 'will it be called tomorrow', 'how far is it to the sun', 'is it a full moon', 'how far is it from the earth to the moon', "when's it gonna snow", 'how far is it to home', 'what time does it get dark', 'how cold is it outside', 'how far is it', 'how long does it take to boil an egg', 'how much is it', 'is it a question?', 'What is it you really wanted to know ?', "it didn't make sense not to live for fun", "it's a cool place, and they say it gets cooler", "it's a cool place, and they say it gets cooler", "that's the way

#### language other than english

In [61]:
#  https://pypi.org/project/langdetect/
from collections import defaultdict
from langdetect import detect

lang_req = defaultdict(list)
non_alpha = []

for line in preprocessed_un_req:
    try:
        if len(line) > 0:
            lang = detect(line)
            lang_req[lang].append(line)
    except:
        non_alpha.append(line)

In [62]:
for k,v in lang_req.items():
    print(f'{k}!!!!!!: {v}')

en!!!!!!: ['Who is George MacDonald Church ?', 'Who is Macron ?', 'What is the plot of Titanic?', 'who is sachin>', 'what is anthem of india?', 'what is life', 'How many bones does a cat have?', 'How to care a newborn?', 'How to care for newborn?', 'Tell me something about Rachmaninov', 'what is konami', 'where was george clooney born?', 'who is the mayor of Varese?', 'What is the German Empire?', 'how many members does the Beatles have?', 'The top biggest cities', 'The most difficult problem', 'what is apple', 'Abbeville County', 'which country has the largest population', 'what is the population of earth?', 'what is a green cloudy object in space discovered by a girl in the netherlands', 'green space cloud', 'what year did star wars come out?', 'who wrote the harry potter books', 'Who wrote the harry potter books?', 'What will you eat for supper today?', 'You may help me choosing a recipe for supper', 'Is Max Planck a man?', 'where i am', 'how old is bruno mars', 'how old bruno mars 

In [64]:
print(non_alpha)

['123123', '...', ':)', '?', '1', '1', '1', '12 12', '....']
